In [1]:
from keras.models import load_model
import keras.losses
import keras.metrics

from trainer.constants import IM_SHAPE_BBOX
from trainer.data import load_train_data, bounding_box, normalize_bounding_box, original_bounding_box
from trainer.utils import bbox_loss_batch, bbox_mIOU

Using TensorFlow backend.


In [2]:
keras.losses.inner_loss = bbox_loss_batch(1, 5, 0.1)
keras.metrics.bbox_mIOU = bbox_mIOU
model = load_model('custom_cnn_model.h5')

In [4]:
X_train, Y_train_orig = load_train_data(resize_shape=IM_SHAPE_BBOX)
Y_train_unscaled = bounding_box(Y_train_orig)
Y_train = normalize_bounding_box(Y_train_unscaled)

In [5]:
res = model.evaluate(x=X_train, y=Y_train)

4000/4000 [==============================] - 33s 8ms/step


In [8]:
print('[loss, mIOU]')
print(res)

[loss, mIOU]
[0.31758892560005186, 0.19524757075309754]


In [9]:
Y_train_pred = model.predict(X_train)

In [21]:
ix=12
print(Y_train[ix])
print(Y_train_pred[ix])

[1.         0.1875     0.10546875 0.375      0.2109375 ]
[0.95818377 0.3897212  0.54383224 0.6339289  0.6725404 ]


In [10]:
import tensorflow as tf
tf.reset_default_graph()
with tf.Session() as sess:
    # sess = tf_debug.TensorBoardDebugWrapperSession(sess, 'DESKTOP-QL1F43N:6064')
    labels = tf.placeholder(tf.float32, (None, 5), name='labels')
    predictions = tf.placeholder(tf.float32, [None, 5], name='predictions')
    iou = bbox_mIOU(labels, predictions)
    sess.run(tf.global_variables_initializer())
    out = sess.run([iou], feed_dict={
        labels: Y_train[ix:ix+1],
        predictions: [[1,  0.86757857, 0.6710704,  0.85394347, 1.3460906 ]]
    })
    print(out)

[0.41362816]
